<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250618/%E3%83%8A%E3%83%83%E3%83%97%E3%82%B6%E3%83%83%E3%82%AF%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E3%81%AE%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **準備**
まず、問題を解くために必要な `python-mip` パッケージをインストールする。

In [ ]:
%%capture
# python-mip パッケージのインストール
!pip install mip

---

### **演習問題1**

#### 1. 問題の確認
基本的な0-1ナップザック問題である。与えられたアイテムのリストとナップザックの容量に基づき、ナップザックに入れるアイテムの総価値が最大になる組み合わせを求める。

-   **アイテム**:
    -   価値: `[45, 30, 50, 20, 65, 40, 25, 70]`
    -   重さ: `[5, 3, 6, 2, 7, 4, 3, 8]`
-   **ナップザック容量**: 15

#### 2. 数理モデルの定式化
本問題は、どのアイテムをナップザックに入れるか（1）、入れないか（0）を決定する0-1整数計画問題として定式化できる。

-   **決定変数**:
アイテム $i$ をナップザックに入れるとき $x_i = 1$、入れないとき $x_i = 0$ となるバイナリ変数を定義する ($i=0, \dots, 7$)。
$$
    x_i \in \{0, 1\}, \quad \forall i \in \{0, 1, \dots, 7\}
$$

-   **目的関数**:
    ナップザック内のアイテムの総価値を最大化する。
    $$
    \text{Maximize} \quad Z = 45x_0 + 30x_1 + 50x_2 + 20x_3 + 65x_4 + 40x_5 + 25x_6 + 70x_7
    $$

-   **制約条件**:
    選択したアイテムの総重量がナップザックの容量を超えてはならない。
    $$
    5x_0 + 3x_1 + 6x_2 + 2x_3 + 7x_4 + 4x_5 + 3x_6 + 8x_7 \le 15
    $$

#### 3. Python (MIP) による実装
`python-mip` を使用して上記モデルを実装し、最適解を求める。

In [ ]:
import mip

# 問題設定
values = [45, 30, 50, 20, 65, 40, 25, 70] # 価値
weights = [5, 3, 6, 2, 7, 4, 3, 8]      # 重さ
capacity = 15                          # ナップザック容量
num_items = len(values)

# 1. モデルの作成
# 最大化問題としてモデルインスタンスを作成する。
model = mip.Model(name="Knapsack_Ex1", sense=mip.MAXIMIZE)

# 2. 変数の定義
# 各アイテムを入れるか(1)入れないか(0)を表すバイナリ変数をリストとして定義する。
x = [model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in range(num_items)]

# 3. 目的関数の設定
# 選択されたアイテムの価値の合計を最大化する。
model.objective = mip.xsum(values[i] * x[i] for i in range(num_items))

# 4. 制約条件の追加
# 選択されたアイテムの重さの合計がナップザックの容量を超えないようにする。
model.add_constr(mip.xsum(weights[i] * x[i] for i in range(num_items)) <= capacity, name="capacity_constraint")

# 5. 求解
# 最適化を実行する。
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print("--- 演習問題1 解答 ---")
    print(f"最大総価値: {model.objective_value:.1f}")

    total_weight = 0
    print("\n選択されたアイテム:")
    for i in range(num_items):
        if x[i].x >= 0.99:
            print(f"  アイテム {i}: 価値={values[i]}, 重さ={weights[i]}")
            total_weight += weights[i]
    print(f"\n選択されたアイテムの総重量: {total_weight:.1f} (容量: {capacity})")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

### **演習問題2**

#### 1. 問題の確認
各アイテムをナップザックの容量が許す限り何個でも選択できるナップザック問題（個数無制限ナップザック問題）を解く。

-   **アイテム**:
    -   価値: `[10, 40, 30, 50]`
    -   重さ: `[5, 4, 6, 3]`
-   **ナップザック容量**: 10

#### 2. 数理モデルの定式化
本問題では、決定変数が0以上の整数を取る整数計画問題となる。

-   **決定変数**:
アイテム $i$ を選択する個数を表す非負**整数**変数 $x_i$ を定義する ($i=0, \dots, 3$)。
$$
    x_i \ge 0, \quad x_i \in \mathbb{Z}, \quad \forall i \in \{0, 1, 2, 3\}
$$

-   **目的関数**:
    総価値を最大化する。
    $$
    \text{Maximize} \quad Z = 10x_0 + 40x_1 + 30x_2 + 50x_3
    $$

-   **制約条件**:
    総重量がナップザックの容量を超えてはならない。
    $$
    5x_0 + 4x_1 + 6x_2 + 3x_3 \le 10
    $$

#### 3. Python (MIP) による実装
変数の型を `BINARY` から `INTEGER` に変更して対応する。

In [ ]:
import mip

# 問題設定
values = [10, 40, 30, 50] # 価値
weights = [5, 4, 6, 3]   # 重さ
capacity = 10            # ナップザック容量
num_items = len(values)

# 1. モデルの作成
model = mip.Model(name="Unbounded_Knapsack_Ex2", sense=mip.MAXIMIZE)

# 2. 変数の定義
# 各アイテムを選択する個数を表す整数変数(Integer)を定義する。
# 下限はデフォルトで0なので、lb=0.0 の指定は省略が可能。
x = [model.add_var(var_type=mip.INTEGER, name=f"x_{i}") for i in range(num_items)]

# 3. 目的関数の設定
model.objective = mip.xsum(values[i] * x[i] for i in range(num_items))

# 4. 制約条件の追加
model.add_constr(mip.xsum(weights[i] * x[i] for i in range(num_items)) <= capacity, name="capacity_constraint")

# 5. 求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print("--- 演習問題2 解答 ---")
    print(f"最適化成功！")
    print(f"最大総価値: {model.objective_value:.1f}")

    total_weight = 0
    print("\n選択されたアイテムと個数:")
    for i in range(num_items):
        if x[i].x > 0.001: # 0より大きい個数のアイテムを表示
            print(f"  アイテム {i} (価値={values[i]}, 重さ={weights[i]}): {int(round(x[i].x))} 個")
            total_weight += weights[i] * x[i].x
    print(f"\n選択されたアイテムの総重量: {total_weight:.1f} (容量: {capacity})")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

---

### **演習問題3**

#### 1. 問題の確認
目的関数と制約条件が標準的なナップザック問題とは異なる。
総価値が指定された目標値以上となる組合せの中で、総重量が最小となるものを求める。

-   **アイテム**:
    -   価値: `[70, 80, 100, 60, 90]`
    -   重さ: `[10, 12, 18, 9, 14]`
-   **目標総価値**: 200

#### 2. 数理モデルの定式化
本問題は、目的が総重量の最小化となり、制約が目標総価値の達成となる。

-   **決定変数**:
アイテム $i$ を選択するかどうかを表すバイナリ変数 $x_i$ を定義する ($i=0, \dots, 4$)。
$$
    x_i \in \{0, 1\}, \quad \forall i \in \{0, 1, 2, 3, 4\}
$$

-   **目的関数**:
選択したアイテムの総重量を最小化する。
$$
    \text{Minimize} \quad W = 10x_0 + 12x_1 + 18x_2 + 9x_3 + 14x_4
$$

-   **制約条件**:
    選択したアイテムの総価値が目標値以上でなければならない。
    $$
    70x_0 + 80x_1 + 100x_2 + 60x_3 + 90x_4 \ge 200
    $$

#### 3. Python (MIP) による実装
`sense`を`mip.MINIMIZE`に設定し、目的関数と制約条件を上記モデルに合わせて変更する。

In [ ]:
import mip

# 問題設定
values = [70, 80, 100, 60, 90]  # 価値
weights = [10, 12, 18, 9, 14]   # 重さ
min_value_req = 200             # 目標総価値
num_items = len(values)

# 1. モデルの作成
# 最小化問題としてモデルインスタンスを作成する。
model = mip.Model(name="Minimize_Weight_Ex3", sense=mip.MINIMIZE)

# 2. 変数の定義
x = [model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in range(num_items)]

# 3. 目的関数の設定
# 選択されたアイテムの重さの合計を最小化する。
model.objective = mip.xsum(weights[i] * x[i] for i in range(num_items))

# 4. 制約条件の追加
# 選択されたアイテムの価値の合計が目標総価値以上になるようにする。
model.add_constr(mip.xsum(values[i] * x[i] for i in range(num_items)) >= min_value_req, name="value_requirement")

# 5. 求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print("--- 演習問題3 解答 ---")
    print(f"最適化成功！")
    print(f"最小総重量: {model.objective_value:.1f}")

    total_value = 0
    print("\n選択されたアイテム:")
    for i in range(num_items):
        if x[i].x >= 0.99:
            print(f"  アイテム {i}: 価値={values[i]}, 重さ={weights[i]}")
            total_value += values[i]
    print(f"\n選択されたアイテムの総価値: {total_value:.1f} (目標値: {min_value_req})")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

---

### **演習問題4**

#### 1. 問題の確認
アイテムの選択に依存関係があるナップザック問題。特定のアイテム間の選択ルールが追加制約として加わる。

-   **アイテム**:
    -   価値: `[50, 80, 60, 70, 40]`
    -   重さ: `[5, 7, 4, 6, 3]`
-   **ナップザック容量**: 15
-   **追加制約**:
    1.  アイテム0を選択する場合、アイテム2も必ず選択しなければならない。
    2.  アイテム1とアイテム3は同時には選択できない。

#### 2. 数理モデルの定式化
標準的な0-1ナップザック問題のモデルに、追加の論理制約を線形不等式で表現して加える。

-   **決定変数**:
アイテム $i$ を選択するかどうかを表すバイナリ変数 $x_i$ を定義する ($i=0, \dots, 4$)。
$$
    x_i \in \{0, 1\}, \quad \forall i \in \{0, 1, \dots, 4\}
$$

-   **目的関数**:
総価値を最大化する。
$$
    \text{Maximize} \quad Z = 50x_0 + 80x_1 + 60x_2 + 70x_3 + 40x_4
$$

-   **制約条件**:
    1.  **容量制約**: 総重量は容量を超えない。
        $$
        5x_0 + 7x_1 + 4x_2 + 6x_3 + 3x_4 \le 15
        $$
    2.  **依存制約1**: アイテム0を選択($x_0=1$)するなら、アイテム2も選択($x_2=1$)する。これは $x_0 \le x_2$ と表現できる。
        $$
        x_0 - x_2 \le 0
        $$
    3.  **依存制約2**: アイテム1とアイテム3は同時に選択できない。これは $x_1 + x_3 \le 1$ と表現できる。
        $$
        x_1 + x_3 \le 1
        $$

#### 3. Python (MIP) による実装
`model.add_constr()` を用いて、上記の追加制約をモデルに組み込む。

In [ ]:
import mip

# 問題設定
values = [50, 80, 60, 70, 40] # 価値
weights = [5, 7, 4, 6, 3]   # 重さ
capacity = 15               # ナップザック容量
num_items = len(values)

# 1. モデルの作成
model = mip.Model(name="Conditional_Knapsack_Ex4", sense=mip.MAXIMIZE)

# 2. 変数の定義
x = [model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in range(num_items)]

# 3. 目的関数の設定
model.objective = mip.xsum(values[i] * x[i] for i in range(num_items))

# 4. 制約条件の追加
# 容量制約
model.add_constr(mip.xsum(weights[i] * x[i] for i in range(num_items)) <= capacity, name="capacity_constraint")

# 追加制約1: アイテム0を選択するならアイテム2も選択
# x[0] <= x[2]  -->  x[0] - x[2] <= 0
model.add_constr(x[0] - x[2] <= 0, name="dependency1")

# 追加制約2: アイテム1とアイテム3は同時に選択不可
# x[1] + x[3] <= 1
model.add_constr(x[1] + x[3] <= 1, name="dependency2")

# 5. 求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print("--- 演習問題4 解答 ---")
    print(f"最適化成功！")
    print(f"最大総価値: {model.objective_value:.1f}")

    total_weight = 0
    print("\n選択されたアイテム:")
    for i in range(num_items):
        if x[i].x >= 0.99:
            print(f"  アイテム {i}: 価値={values[i]}, 重さ={weights[i]}")
            total_weight += weights[i]
    print(f"\n選択されたアイテムの総重量: {total_weight:.1f} (容量: {capacity})")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

---

### **演習問題5**

#### 1. 問題の確認
重さに加えて「体積」という第二の制約が加わった多次元ナップザック問題。

-   **アイテム**:
    -   価値: `[100, 150, 80, 120, 90, 130]`
    -   重さ: `[10, 15, 7, 12, 8, 14]`
    -   体積: `[5, 8, 6, 7, 4, 9]`
-   **ナップザック容量**:
    -   最大許容重量: 30
    -   最大許容体積: 20

#### 2. 数理モデルの定式化
制約条件が「重さ」と「体積」の2つになる。

-   **決定変数**:
アイテム $i$ を選択するかどうかを表すバイナリ変数 $x_i$ を定義する ($i=0, \dots, 5$)。
$$
    x_i \in \{0, 1\}, \quad \forall i \in \{0, 1, \dots, 5\}
$$

-   **目的関数**:
    総価値を最大化する。
    $$
    \text{Maximize} \quad Z = 100x_0 + 150x_1 + 80x_2 + 120x_3 + 90x_4 + 130x_5
    $$

-   **制約条件**:
    1.  **重量制約**: 総重量が最大許容重量を超えない。
        $$
        10x_0 + 15x_1 + 7x_2 + 12x_3 + 8x_4 + 14x_5 \le 30
        $$
    2.  **体積制約**: 総体積が最大許容体積を超えない。
        $$
        5x_0 + 8x_1 + 6x_2 + 7x_3 + 4x_4 + 9x_5 \le 20
        $$

#### 3. Python (MIP) による実装
2つの制約をモデルに追加する。

In [ ]:
import mip

# 問題設定
values = [100, 150, 80, 120, 90, 130] # 価値
weights = [10, 15, 7, 12, 8, 14]     # 重さ
volumes = [5, 8, 6, 7, 4, 9]         # 体積
capacity_weight = 30                 # 最大許容重量
capacity_volume = 20                 # 最大許容体積
num_items = len(values)

# 1. モデルの作成
model = mip.Model(name="Multidimensional_Knapsack_Ex5", sense=mip.MAXIMIZE)

# 2. 変数の定義
x = [model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in range(num_items)]

# 3. 目的関数の設定
model.objective = mip.xsum(values[i] * x[i] for i in range(num_items))

# 4. 制約条件の追加
# 重量制約
model.add_constr(mip.xsum(weights[i] * x[i] for i in range(num_items)) <= capacity_weight, name="weight_capacity")

# 体積制約
model.add_constr(mip.xsum(volumes[i] * x[i] for i in range(num_items)) <= capacity_volume, name="volume_capacity")

# 5. 求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print("--- 演習問題5 解答 ---")
    print(f"最適化成功！")
    print(f"最大総価値: {model.objective_value:.2f}")

    total_weight = 0
    total_volume = 0
    print("\n選択されたアイテム:")
    for i in range(num_items):
        if x[i].x >= 0.99:
            print(f"  アイテム {i}: 価値={values[i]}, 重さ={weights[i]}, 体積={volumes[i]}")
            total_weight += weights[i]
            total_volume += volumes[i]

    print(f"\n選択されたアイテムの総重量: {total_weight:.2f} (容量: {capacity_weight})")
    print(f"選択されたアイテムの総体積: {total_volume:.2f} (容量: {capacity_volume})")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

---

### **応用問題6**

#### 1. 問題の確認
アイテムがカテゴリに分類されており、各カテゴリから少なくとも1つのアイテムを選択するという追加制約を持つナップザック問題。

-   **アイテム**:
    -   価値: `[30, 20, 50, 70, 40, 60, 80]`
    -   重さ: `[2, 3, 5, 6, 3, 4, 7]`
    -   カテゴリID: `[0, 0, 1, 1, 1, 2, 2]`
-   **ナップザック容量**: 15
-   **追加制約**:
    -   カテゴリ0 (アイテム0, 1) から少なくとも1つ選択する。
    -   カテゴリ1 (アイテム2, 3, 4) から少なくとも1つ選択する。
    -   カテゴリ2 (アイテム5, 6) から少なくとも1つ選択する。

#### 2. 数理モデルの定式化
カテゴリごとの選択制約を線形不等式で表現する。

-   **決定変数**:
アイテム $i$ を選択するかどうかを表すバイナリ変数 $x_i$ を定義する ($i=0, \dots, 6$)。
$$
    x_i \in \{0, 1\}, \quad \forall i \in \{0, 1, \dots, 6\}
$$

-   **目的関数**:
    総価値を最大化する。
$$
    \text{Maximize} \quad Z = 30x_0 + 20x_1 + 50x_2 + 70x_3 + 40x_4 + 60x_5 + 80x_6
$$

-   **制約条件**:
    1.  **容量制約**:
        $$
        2x_0 + 3x_1 + 5x_2 + 6x_3 + 3x_4 + 4x_5 + 7x_6 \le 15
        $$
    2.  **カテゴリ0制約**: アイテム0または1の少なくとも一方が選ばれる。
        $$
        x_0 + x_1 \ge 1
        $$
    3.  **カテゴリ1制約**: アイテム2, 3, 4の少なくとも一つが選ばれる。
        $$
        x_2 + x_3 + x_4 \ge 1
        $$
    4.  **カテゴリ2制約**: アイテム5または6の少なくとも一方が選ばれる。
        $$
        x_5 + x_6 \ge 1
        $$

#### 3. Python (MIP) による実装
カテゴリ分類に基づいて、それぞれのカテゴリに対する制約を追加する。

In [ ]:
import mip

# 問題設定
values = [30, 20, 50, 70, 40, 60, 80] # 価値
weights = [2, 3, 5, 6, 3, 4, 7]      # 重さ
# カテゴリID: [0, 0, 1, 1, 1, 2, 2]
capacity = 15                        # ナップザック容量
num_items = len(values)

# 1. モデルの作成
model = mip.Model(name="Categorized_Knapsack_Ex6", sense=mip.MAXIMIZE)

# 2. 変数の定義
x = [model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in range(num_items)]

# 3. 目的関数の設定
model.objective = mip.xsum(values[i] * x[i] for i in range(num_items))

# 4. 制約条件の追加
# 容量制約
model.add_constr(mip.xsum(weights[i] * x[i] for i in range(num_items)) <= capacity, name="capacity_constraint")

# カテゴリ制約
# カテゴリ0 (アイテム 0, 1) から少なくとも1つ
model.add_constr(x[0] + x[1] >= 1, name="category0_req")

# カテゴリ1 (アイテム 2, 3, 4) から少なくとも1つ
model.add_constr(x[2] + x[3] + x[4] >= 1, name="category1_req")

# カテゴリ2 (アイテム 5, 6) から少なくとも1つ
model.add_constr(x[5] + x[6] >= 1, name="category2_req")

# 5. 求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print("--- 応用問題6 解答 ---")
    print(f"最適化成功！")
    print(f"最大総価値: {model.objective_value:.1f}")

    total_weight = 0
    print("\n選択されたアイテム:")
    for i in range(num_items):
        if x[i].x >= 0.99:
            print(f"  アイテム {i}: 価値={values[i]}, 重さ={weights[i]}")
            total_weight += weights[i]

    print(f"\n選択されたアイテムの総重量: {total_weight:.1f} (容量: {capacity})")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")